In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.io import arff
import os
import scipy.stats as stats
from scipy.stats import lognorm
from scipy.stats import norm
from joblib import Parallel, delayed
from tqdm import tqdm

SEED = 14208

In [22]:
import warnings
warnings.filterwarnings("ignore")

In [23]:
dataset_filepath = 'datasets/house_8L.arff'  # Aquí se incluirían las rutas a los datasets
pred_col_name = 'price'  # Columna a predecir
alpha = 0.05  # Nivel de significancia

In [24]:
def process_dataset(dataset_filepath: str, pred_col_name: str):
    # Obtener la extension del archivo
    _, file_extension = os.path.splitext(dataset_filepath)

    # Cargar el dataset según la extensión
    if file_extension == '.arff':
        data = arff.loadarff(dataset_filepath)
        df = pd.DataFrame(data[0])

    elif file_extension == '.csv':
        df = pd.read_csv(dataset_filepath)

    else:
        raise ValueError("Formato no soportado, `dataset_filepath` debe tener una de las siguientes extensiones: .csv, .arff")

    # Separar el dataset en Train y Validation
    train_df, validation_df = train_test_split(df, test_size=0.2, random_state=SEED)
    y_train, y_valid = train_df[pred_col_name], validation_df[pred_col_name]
    X_train, X_valid = train_df.drop(pred_col_name, axis=1), validation_df.drop(pred_col_name, axis=1)

    # Aplicar get_dummies para variables categóricas
    X_train = pd.get_dummies(X_train)
    X_valid = pd.get_dummies(X_valid)
    X_train, X_valid = X_train.align(X_valid, join='left', axis=1, fill_value=0) 

    # Crear y entrenar el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=SEED)
    rf_model.fit(X_train, y_train)

    # Obtener las predicciones de cada árbol en el bosque
    tree_predictions = []
    for tree in rf_model.estimators_:
        tree_pred = tree.predict(X_valid)
        tree_predictions.append(tree_pred)

    tree_predictions = np.array(tree_predictions).T

    # Evaluar el modelo
    predictions = rf_model.predict(X_valid)
    mse = mean_squared_error(y_valid, predictions)

    return df, mse, predictions, tree_predictions

In [25]:
def lognormal_KS_fit_check(tree_predictions, shift=1e-6, alpha=0.05, orig_data_stat=0.0722):
    ks_results = []
    tree_predictions += shift
    for predictions in tree_predictions:
        # Estimate parameters of the lognormal distribution
        shape, loc, scale = stats.lognorm.fit(predictions, floc=0)
        
        # Perform the Kolmogorov-Smirnov test
        ks_stat, p_value = stats.kstest(predictions, 'lognorm', args=(shape, loc, scale))
        ks_results.append((ks_stat, p_value, shape, loc, scale))

    # Proportion of trees that fit the lognormal distribution
    ks_stats = [result[0] for result in ks_results]
    eps = alpha * orig_data_stat
    filtered_stats = [stat for stat in ks_stats if orig_data_stat-eps <= stat <= orig_data_stat+eps]
    ks_fit_proportion = len(filtered_stats) / len(ks_stats)
    
    return ks_fit_proportion, ks_stats

In [26]:
def gamma_KS_fit_check(tree_predictions, shift=1e-6, alpha=0.05, orig_data_stat=0.1323):
    ks_results = []
    tree_predictions += shift
    for predictions in tree_predictions:
        # Estimate parameters of the lognormal distribution
        shape, loc, scale = stats.gamma.fit(predictions, floc=0)
        
        # Perform the Kolmogorov-Smirnov test
        ks_stat, p_value = stats.kstest(predictions, 'gamma', args=(shape, loc, scale))
        ks_results.append((ks_stat, p_value, shape, loc, scale))

    # Proportion of trees that fit the lognormal distribution
    ks_stats = [result[0] for result in ks_results]
    eps = alpha * orig_data_stat
    filtered_stats = [stat for stat in ks_stats if orig_data_stat-eps <= stat <= orig_data_stat+eps]
    ks_fit_proportion = len(filtered_stats) / len(ks_stats)
    
    return ks_fit_proportion

In [27]:
def transforme_normal_SW_fit_check(tree_predictions, shift=1e-6, alpha=0.05, orig_data_stat=0.9809):
    sw_results = []
    tree_predictions = np.log1p(tree_predictions)

    for predictions in tree_predictions:
        # Perform the Shapiro-Wilk test
        shapiro_stat, shapiro_p_value = stats.shapiro(predictions)
        sw_results.append((shapiro_stat, shapiro_p_value))

    # Proportion of trees that fit the normal distribution
    sw_stats = [result[0] for result in sw_results]
    eps = alpha * orig_data_stat
    filtered_stats = [stat for stat in sw_stats if orig_data_stat-eps <= stat <= orig_data_stat+eps]
    sw_fit_proportion = len(filtered_stats) / len(sw_stats)
    
    return sw_fit_proportion

In [28]:
dataset_titanic, mse_titanic, predictions_titanic, tree_predictions_titanic = process_dataset(dataset_filepath, pred_col_name)

ks_lognormal_fit_proportion, ks_stats = lognormal_KS_fit_check(tree_predictions_titanic, alpha=alpha)
print(f"Proportion of trees that fit the lognormal distribution: {ks_lognormal_fit_proportion:.5f}")

sw_fit_proportion = transforme_normal_SW_fit_check(tree_predictions_titanic, alpha=alpha)
print(f"Proportion of trees that fit the normal distribution (over transformed data): {sw_fit_proportion:.5f}")

Proportion of trees that fit the lognormal distribution: 0.02326
Proportion of trees that fit the normal distribution (over transformed data): 0.48212


In [29]:
dataset_name = os.path.basename(dataset_filepath)
file_path = 'datasets_results.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Open the file
    df = pd.read_csv(file_path)
else:
    # Create a new dataframe with column headers
    df = pd.DataFrame(columns=['dataset_name', 'prop_lognormal', 'prop_normal', 'alpha'])

# Add a new row with values
new_row = {'dataset_name': dataset_name, 'prop_lognormal': ks_lognormal_fit_proportion, 'prop_normal': sw_fit_proportion, 'alpha': alpha}
df = df.append(new_row, ignore_index=True)

# Save the dataframe to the file
df.to_csv(file_path, index=False)

In [30]:
df

,dataset_name,prop_lognormal,prop_normal,alpha
0,titanic_fare_test.arff,0.003817,0.206107,0.05
1,medical_costs.csv,0.000000,0.007463,0.05
2,salary_football.csv,0.000000,0.223785,0.05
3,flight_price.csv,0.000468,0.038372,0.05
4,Height.csv,0.000000,0.080214,0.05
5,wine_quality.arff,0.000000,0.005385,0.05
6,house_8L.arff,0.023261,0.482115,0.05
